In [0]:
df=spark.read.format('csv').option('header',True).option('infereschema',True).load('/Volumes/practice/default/practice/BigMart Sales.csv')

In [0]:
df.display()

String Functions

In [0]:
from pyspark.sql.functions import*

In [0]:
df.select(initcap("Item_Type")).display()

In [0]:
df.select(upper("Item_Type")).display()
df.select(lower("Item_Type")).display()

Union

In [0]:
data=[("Mani",21),("Ravi",22)]
schema=["name","age"]

df1=spark.createDataFrame(data,schema)

In [0]:
df1.display()

In [0]:
data1=[("Mani",21),("Ravi",22)]
schema1=["name","age"]
df2=spark.createDataFrame(data1,schema1)
df2.display()

In [0]:
df1.unionByName(df2).display()

Date Functions

In [0]:
df=df.withColumn("Current Date",current_date())

In [0]:
df=df.withColumn("Weekafter", date_add(current_date(),7))
df=df.withColumn("Weekbefore",date_sub(current_date(),7))

In [0]:
df.display()

In [0]:
df=df.withColumn("datedifference",date_diff("Current Date","Weekbefore"))

In [0]:
df.display()

In [0]:
df = df.withColumn("Weekbefore", date_format("Weekbefore","MM-yyyy-dd" ))


In [0]:
df.display()

Dropping Nulls

In [0]:
df.dropna("any").display()

In [0]:
df.dropna("all").display()

In [0]:
df.dropna(subset=["Outlet_Size"]).display()

### Filling Nulls

In [0]:
df.fillna("Not Available").display()

In [0]:
df.fillna("Not Available",subset=["Outlet_Size"]).display()

## Split and Indexing

In [0]:
df.withColumn("Outlet_Type",split("Outlet_Type"," ")[1]).display()

Split

In [0]:
dfduplicate=df.withColumn("Outlet_Type",split("Outlet_Type"," "))

In [0]:
dfduplicate.display()

## Array Contains

In [0]:
dfduplicate.withColumn("Type1_flag",array_contains("Outlet_Type","Type1")).display()

##Explode

In [0]:
dfs=df.withColumn("Outlet_Type",split("Outlet_Type"," "))
dfs=dfs.withColumn("Outlet_Type",explode("Outlet_Type"))
dfs.display()

In [0]:
dfs.groupBy("Item_Identifier").agg(collect_list("Outlet_Type")).display()

## Group By

In [0]:
df.display()

In [0]:
df.groupBy("Item_Type").agg(sum("Item_MRP")).display()

In [0]:
df.groupBy("Item_Type").agg(avg("Item_MRP")).display()

In [0]:
df.groupBy("Item_Type", "Outlet_Size").agg(
    avg("Item_MRP").alias("avg_mrp"),
    sum("Item_MRP").alias("sum_mrp")
).display()

## Collect List

In [0]:
data=[("user1","Book1"),("user1","Book2"),("user2","Book1"),("user2","Book2")]
Schema=["User","Book"]
dfsd=spark.createDataFrame(data,Schema)
dfsd.display()

In [0]:
dfsd.groupBy("User").agg(collect_set("Book")).display()

In [0]:
df.display()

### Pivot


In [0]:
df.select("Item_Type","Outlet_Size","Item_MRP").display()


In [0]:
from pyspark.sql.functions import*
df.groupBy("Item_Type").pivot("Outlet_Size").agg(avg("Item_MRP")).display()

### When-Otherwise

In [0]:
from pyspark.sql.functions import*
dfdis=df.withColumn("Veg or Non Veg",when(col("Item_Type")=="Meat","Non-Veg").otherwise("Veg")).display()

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import*

df = df.withColumn(
    "Veg Price Expensive",
    when(
        (col("Veg Price Flag") == "Veg") & (col("Item_Weight").cast("double") > 10), 
        "Veg ExpensiveItem"
    ).when(
        (col("Veg Price Flag") == "Veg") & (col("Item_Weight").cast("double") < 10), 
        "Veg inExpensiveItem"
    ).otherwise("Non-Veg")
)
display(df)

# Joins

In [0]:
data1 = [
    ('1', 'gaur', 'd01'),
    ('2', 'kit', 'd02'),
    ('3', 'sam', 'd03'),
    ('4', 'tim', 'd03'),
    ('5', 'aman', 'd05'),
    ('6','nad','d06')
]

schema1 = 'emp_id STRING, emp_name STRING, dept_id STRING'

df1 = spark.createDataFrame(data1, schema1)

df1.display()

data2 = [
    ('d01', 'HR'),
    ('d02', 'Marketing'),
    ('d03', 'Accounts'),
    ('d04', 'IT'),
    ('d05', 'Finance')
]

schema2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(data2, schema2)

df2.display()


In [0]:
df1.join(df2,df1["dept_id"]==df2["dept_id"],"inner").display()

In [0]:
df1.join(df2, df1["dept_id"] == df2["dept_id"] , "left").display()

In [0]:
df1.join(df2,df1["dept_id"]==df2["dept_id"],"right").display()

In [0]:
df1.join(df2,df1["dept_id"]==df2["dept_id"],"anti").display()